In [5]:
import tensorflow as tf
import numpy as np

def random_X(seq_length=4, vec_dim=4):
    return [[0]+np.random.choice([0, 1], vec_dim).tolist() for i in range(seq_length)]

## 7-2. tensorflowでRNNをカスタマイズする方法

その前に、自分独自のリカレントニューラルネットワークをtensorflowで作る方法を紹介します。tensorflow実装での説明ですから、他のライブラリだと異なる方法を採るかも知れませんが、基本となる考え方は同じです。

既に少し触れたように、tensorflowではRNNを
* `cell = tf.keras.layers.Layer()オブジェクト`として
* `RNN_Layer = tf.keras.layers.RNN(cell)`とする

の2ステップを踏んで作るように設定されているようです。2つめの`tf.keras.layers.RNN(cell)`は勝手にこれまでのようなRNNを作るための処理をしてくれるということで、自分でカスタマイズする場合の主な目標は**cellオブジェクトを作る**ことだと言えます。

### Cell で何を実装すればよいか
Cellでは各時刻で使う訓練パラメータと各時刻での処理を定義します。

![alt](2.jpg)

図に描いたように、`call()`という関数で時刻 t での処理を定義します。内部状態はリストで取り扱うようです。それと`@property`でデコレートした、内部状態の次元のリストを返す関数`state_size()`も定義しておく必要があるようです$^※$。`bulid()`はいらないかもしれません。

> $^※$ `state_size()`について、ベクトルじゃなくてテンソル量を内部状態にしたい場合は、スカラー値ではなく`tf.TensorShape([dim_A, dim_B, ...])`をリストの要素として書けば良いようです。この処理を後で使います。



### 実装してみる
リカレントニューラルネットワークとして最も単純な

![alt](3.jpg)

のようなRNNレイヤー(MySimpleRNN)のためのCellを作るとズバリ以下のようになります：

In [3]:
class MySimpleRNNCell(tf.keras.layers.Layer):
    def __init__(self, units):
        super(MySimpleRNNCell, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.W= tf.keras.layers.Dense(self.units)
        super(MySimpleRNNCell, self).build(input_shape[-1])
        self.build = True
        
    def call(self, inputs, states):
        ''' inputs:xt, states:[h{t-1}], each vector shape = [batchsize, vec_dim]'''
        x, h_prev = inputs, states[0]
        xh = tf.concat([x, h_prev], axis=1)
        h = tf.keras.activations.tanh(self.W(xh))
        return h, [h] # output, next_states
        
    @property
    def state_size(self):
        ''' list of dimensions of states '''
        return [self.units]
    
    def get_initial_state(self, inputs=None, batch_size=None, dtype=None):
        ''' how to initialize states '''
        h0 = tf.zeros([batch_size, self.units], dtype)
        return [h0]

Cellは基本的には
* `tf.keras.layers.Layer`を継承したクラスとして実装
* 内部状態をpythonのリストとして表現する
* `call(self, inputs, states)` で各時系列の時刻$t$における処理を書く。このときの`states`は前の時刻が終わった直後の内部状態リスト
* `@property`でデコレートした関数`state_size(self)`で内部状態の次元のリストを返すようにする
* `get_initial_state`で内部状態の初期化の仕方を指定する

を満たせばどのような実装でもOKです。

### 実際に使う
実際に使う際は、`cell`を`RNN_layer = tf.keras.layers.RNN(cell)` のようにしてラップします。以下のように

![alt](32.jpg)

cellを時系列データ処理の時間方向につないだ縦向きの一つのニューラルネットワーク層を作るのが`tf.keras.layers.RNN()`です。



In [6]:
cell      = MySimpleRNNCell(units=3)
RNN_layer =  tf.keras.layers.RNN(cell, return_sequences=True, return_state=True, stateful=True) # いろいろオプションあり

X = tf.constant([random_X(seq_length=2, vec_dim=10)], dtype=tf.float32)
print("入力\n", X)
print("出力\n", RNN_layer(X))
print("状態\n", RNN_layer.states[0])

入力
 tf.Tensor(
[[[0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0.]]], shape=(1, 2, 11), dtype=float32)
出力
 [<tf.Tensor: shape=(1, 2, 3), dtype=float32, numpy=
array([[[-0.7203732 , -0.7506157 ,  0.60109067],
        [-0.6228929 ,  0.71592975, -0.40907428]]], dtype=float32)>, <tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.6228929 ,  0.71592975, -0.40907428]], dtype=float32)>]
状態
 <tf.Variable 'rnn_1/Variable:0' shape=(1, 3) dtype=float32, numpy=array([[-0.6228929 ,  0.71592975, -0.40907428]], dtype=float32)>


`tf.keras.layers.RNN`でレイヤーを作る際に、いくつかオプションがあります：
* `return_sequences=True/False`: 入力 X の時系列の時刻ごとの出力をするかしないか
* `return_state=True/False`: 内部状態を出力するかしないか(`return_sequences=True`でも、最終時刻の内部状態しか出ないっぽいです)
* `stateful=True/False`: 内部状態を保持するかしないか。保持しない場合は処理毎に初期化(なぜかNone?)される。

より詳しくはtensorflowのドキュメントを読んでいただければと思います。